In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,8, 3, padding=0) #input -? OUtput? RF
        self.bn1=nn.BatchNorm2d(8)
        self.dp1=nn.Dropout(p=0.1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=0)
        self.bn2=nn.BatchNorm2d(16)
        self.dp2=nn.Dropout(p=0.1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16,8, 1, padding=0)
        self.bn3=nn.BatchNorm2d(8)
        self.conv4 = nn.Conv2d(8,16, 3, padding=0)
        self.bn4=nn.BatchNorm2d(16)
        self.dp3=nn.Dropout(p=0.1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(16,32, 3)
        self.bn5=nn.BatchNorm2d(32)
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        #self.conv7 = nn.Conv2d(1024, 10, 3)
        #self.Gpool=nn.AvgPool2d(64)
        self.fc1 = nn.Linear(3*3*32,30)
        self.fc2 = nn.Linear(30, 10)
    def forward(self, x):
        x=F.relu(self.conv1(x))
        x=self.bn1(x)
        x=self.dp1(x)
        x=F.relu(self.conv2(x))
        x=self.bn2(x)
        x=self.dp2(x)
        x=self.pool1(x)
        x=F.relu(self.conv3(x))
        x=self.bn3(x)
        x=F.relu(self.conv4(x))
        x=self.bn4(x)
        x=self.dp3(x)
        x=self.pool2(x)
        x=F.relu(self.conv5(x))
        x=self.bn5(x)
        #x=self.Gpool(x)
        x = x.view(-1, 3*3*32)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.log_softmax(x,dim=1)
   



In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
           Dropout-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,168
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8            [-1, 8, 12, 12]             136
       BatchNorm2d-9            [-1, 8, 12, 12]              16
           Conv2d-10           [-1, 16, 10, 10]           1,168
      BatchNorm2d-11           [-1, 16, 10, 10]              32
          Dropout-12           [-1, 16, 10, 10]               0
    

In [4]:
torch.manual_seed(1)

train_transforms = transforms.Compose([
    transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
    transforms.Resize((28, 28)),
    transforms.RandomRotation((-15., 15.), fill=0),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

train_data = datasets.MNIST('../data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('../data', train=False, download=True, transform=test_transforms)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1, verbose=True)
for epoch in range(0, 20):
    print ("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch:  0


loss=0.09159252047538757 batch_id=234: 100%|██████████| 235/235 [00:55<00:00,  4.24it/s]



Test set: Average loss: 0.1162, Accuracy: 9670/10000 (96.70%)

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch:  1


loss=0.051892708986997604 batch_id=234: 100%|██████████| 235/235 [00:49<00:00,  4.77it/s]



Test set: Average loss: 0.0655, Accuracy: 9804/10000 (98.04%)

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch:  2


...


Test set: Average loss: 0.0367, Accuracy: 9898/10000 (98.98%)

Adjusting learning rate of group 0 to 1.0000e-05.
Epoch:  18


loss=0.018355820327997208 batch_id=234: 100%|██████████| 235/235 [00:48<00:00,  4.81it/s]



Test set: Average loss: 0.0358, Accuracy: 9901/10000 (99.01%)

Adjusting learning rate of group 0 to 1.0000e-05.
Epoch:  19


loss=0.009502561762928963 batch_id=234: 100%|██████████| 235/235 [00:48<00:00,  4.86it/s]



Test set: Average loss: 0.0361, Accuracy: 9899/10000 (98.99%)

Adjusting learning rate of group 0 to 1.0000e-05.
